<a href="https://colab.research.google.com/github/enrii21/frontend-app/blob/main/House_price_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Environment Settings**

This first part is aimed to set our working enviroment to easily download a dataset from Kaggle repositories, and import some useful Python libraries that we will meet later on.

**title something**


In [ ]:
print("2")

2


## **Kaggle install and dataset import**

Here we just open a dialogue to upload our Kaggle API Token, which comes in the forms of a json document.

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! ls -lha kaggle.json
! pip install -q kaggle # installing the kaggle package
! mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
! cp kaggle.json ~/.kaggle/ # move the key to the folder
! pwd # checking the present working directory
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d harlfoxem/housesalesprediction -p /content/input_data/house_sales

In [ ]:
! ls -l input_data/house_sales/housesalesprediction.zip
! unzip input_data/house_sales/housesalesprediction.zip -d input_data/house_sales/

In [ ]:
! pip install pandas numpy matplotlib seaborn scikit-learn

In this assignment, we will use a simplified data and create a simple **linear regression model**. The dataset can be downloaded from https://www.kaggle.com/harlfoxem/housesalesprediction/download.

This dataset contains house sale prices for Kings County, which includes Seattle. It includes homes sold between May 2014 and May 2015. There are several versions of the data. Some additional information about the columns is available here: https://geodacenter.github.io/data-and-lab/KingCounty-HouseSales2015/, some of which are copied below.

* **id**	Identification
* **date**	Date sold
* **price**	Sale price
* **bedrooms**	Number of bedrooms
* **bathrooms**	Number of bathrooms
* **sqft_liv**	Size of living area in square feet
* **sqft_lot**	Size of the lot in square feet
* **floors**	Number of floors
* **waterfront**	‘1’ if the property has a waterfront, ‘0’ if not
* **view**	An index from 0 to 4 of how good the view of the property was
* **condition**	Condition of the house, ranked from 1 to 5
* **grade**	Classification by construction quality which refers to the types of materials used and the quality of workmanship. Buildings of better quality (higher grade) cost more to build per unit of measure and command higher value.
* **sqft_above**	Square feet above ground
* **sqft_basmt**	Square feet below ground
* **yr_built**	Year built
* **yr_renov**	Year renovated. ‘0’ if never renovated
* **zipcode**	5 digit zip code
* **lat**	Latitude
* **long**	Longitude
* **squft_liv15**	Average size of interior housing living space for the closest 15 houses, in square feet
* **squft_lot15**	Average size of land lost for the closest 15 houses, in square feet

In the cells below we deal with all the Python imports that will be useful for our analysis.

In [ ]:
import scipy as sp
import scipy.stats as stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
sns.set()
import statsmodels.formula.api as smf
import statsmodels.api as sm
# Any graph which we are creating as a part of our code must appear in the same notebook and not in separate window
%matplotlib inline

# **Munging Data**

In this part, let's load and inspect data. We will also learn how to transform columns when needed.


> 💡**Tip:** `pd.read_csv(<file path>)` reads a csv file and returns to pandas data frame object. It can also read files with other delimiter such as `.tsv` files. pandas also has `pd.read_excel` to read Excel files. See more in [this document](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html).








In [ ]:
df = pd.read_csv('input_data/house_sales/kc_house_data.csv')

## **Date string to numbers**

Now, let's overview the dataframe. Using `.head()` on the dataframe, we can see the first 5 rows of the data. You can specify number of rows as argument then it will show those number of rows. similarly, `.tail()` gives the last 5 rows by default. You can see the columns names, but not all columns are displayed if there are too many columns.

> 💡**Tip:** If you want to show all columns and rows, there are [pandas command](https://www.geeksforgeeks.org/show-all-columns-of-pandas-dataframe-in-jupyter-notebook/) setting max rows and cols.

The column 'date' is the date sold (with some timestamp as well), and the data is string type. Note that sometimes data tables may have date/time columns as datetime object types. In this example, it has a string type. We will extract year and month information from the string. In the data frame `df`, let's create new features 'sales_year' and 'sales_month' using 'date' column.

> 💡**Tip:** You can use either bracket (e.g. `df['date']`) or dot (e.g. `df.date`) to get the column `'date'` in the data frame `df`. A single columns object from dataframe is a pandas series object type, and you can use `.apply()` method for a transformation. `.apply()` is generic and can be applied to not only to single column (pandas series) but also to multiple columns (pandas dataframe). Here, we will apply it to a single column object and use `lambda` function inside the `.apply()` as shown below. You can find more examples, see the [doc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html), which serves also as a precious reference.

In this case, when we inspect the `'date'` column, it is a string object, so we can slice the year and month from the string. Also, we'd like to convert the extracted year and month strings to ingeters.

In [ ]:
df.head()

With `iloc` (Pandas library), we can select and access data in DataFrames or Series using integer-based indexing. It allows you to specify rows and columns by their numerical indices.

> 💡**Tip:** Try to create a new code cell and select other columns within the `df` DataFrame, just to understand how `iloc` works. See [docs](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html) for more reference.

In [ ]:
print(df.date)
print(type(df.date.iloc[0]))

In [ ]:
# extract year and month info from the string
# create new features 'sales_year' and 'sales_month' in df

df['sales_year'] = df.date.apply(lambda x: int(x[:4]))
df['sales_month'] = df.date.apply(lambda x: int(x[4:6]))

In [ ]:
df.groupby('sales_month')

Now, let's count how many sales occurred in each month and each year. We can use `.groupby()` function to group by 'sales_month' and 'sales_year' as shown below.

> 💡**Tip:** `.groupby()` itself returns an object that doesn't get displayed, hence not a processed dataframe. It is often used with some other aggregation method, such as `.count()`, `.mean()`, `.sum()`, etc.
In the below example, we use `.count()` to count number of sales per each group (e.g. by sales_month). `.groupby()` can also group by multiple columns. This [resource](https://realpython.com/pandas-groupby/) has more explanations and examples, including some useful SQL analogies.

In the following lines, by selecting a specific column after `groupby()`, you can apply `count()` to count non-null entries in that column only.

In [ ]:
print(df.groupby('sales_month')['id'].count())
print(df.groupby('sales_year')['id'].count())

Create a new text cell and answer to the following questions.

**Question 1.** Based on the output from above cell, which month has the most number of sales?

**Question 2.** Which months has instead the least number of sales?

Now, let's have a look at what data type each columns has. We can use .`info()` method on the dataframe object to see the data type. You can see `int64`, `float64` and `object` in our example. `object` can be string type or something else (such as list or other types of objects).

> 💡**Tip:** Note that sometimes raw data is not adequately formatted, so you might see columns that are supposed to be numbers typed as strings. It is **always a good practice** to firstly inspect columns data types and clean them if necessary.

In [ ]:
df.info()

## **Variable types**

**Review Concepts**

✔️ What data types can be considered as a numeric variable?

✔️ What is the difference between ordinal and non-ordinal categorical variables?

> 💡**Tip:** Is binary categorical variable (Yes/No, Male/Female, True/False, Positive/Negative etc) numeric? Why or why not?
>
> How about a variable that has meaning of degree, such as survey/review ratings (very satisfied = 5, satisfied = 4, neutral = 3, disatisfied = 2, very disatisfied = 1)?
>
> Typically it is recommended to treat ordinal categorical variable (which order has meaning- e.g. degree, grades, numbers, severity etc) as numeric variable because a linear regression (or any ML) model can treat that variable (feature) as numbers and can learn a relationship to the t
target variable y.
>
>Also, categorical variables sometimes need to be binarized (which involves to transform the column into multiple binary columns) before using them in a linear regression model (One-Hot Encoding).

Inspect each feature's data type and variable type. What is the best description for the variable type of following features? Update the strings in the Code Cell below to 'numeric' or 'categorical'.

In [ ]:
# uncomment the feaures below and update the strings with 'numeric' or 'categorical'

'''

price = ''
bathrooms = ''
waterfront = ''
grade = ''
zipcode = ''
sales_year = ''

'''

> 💡**Tip:** Make use of `.unique()` method, referring to each column to get a glimpse of the unique values in each fature within the DataFrame.

Try the cell below to check what unique values exist in each column (expect a pretty long output).

In [ ]:
for c in df.columns[2:]:
    print(c, df[c].unique())

## **Drop features**

Let's drop features that are unnecessary. `id` is not a meaningful feature. `date` string has been coded to `sales_month` and `sales_year`, so we can remove `date`. `zipcode` can be also removed as it's hard to include in a linear regression model and the location info is included in the `lat` and `long` (in case we need it). Drop the features `id`, `date`, and `zipcode` and replace the `df`.

> 💡**Tip:** `.drop()` function can drop one or more columns or rows. Some more suggestions how to use it in the [docs](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html).

In [ ]:
# drop unnecessary features, replace df

df = df.drop(columns=['id', 'date', 'zipcode'])

Test that you droppd the features `id`, `date`, and `zipcode` from `df` by inspecting the dataframe.

# **More inspection, Correlation and Pair-Plot**

## **Get correlation matrix on the data frame**

Which feature may be the best predictor of price based on the correlation? Answer as a string value (e.g. best_guess_predictor = 'price' or best_guess_predictor = 'yr_built')

> 💡**Tip:** `.corr() `finction can show correlation matrix from the dataframe. More resource

**Exercise**

✔️ By looking at the correlation matrix, how do you decide which feature is the best predictor?

In [ ]:
df.corr()

In [ ]:
# update best_guess_predictor with a string value (i.e. the feature name)
best_guess_predictor = ''

## **Display the correlation matrix as heat map**

The method `seaborn.heatmap()` can visualize a matrix as a heatmap. The Seaborn library allows you to easily create highly customized visualizations of your data, such as line plots, histograms, and heatmaps.

In order to visualize the correlation matrix using `seaborn.heatmap()`, play with color map, text font size, decimals, text orientation etc. To have an overview of the different parameters to be set, the classic [docs](https://seaborn.pydata.org/generated/seaborn.heatmap.html) and this [cheatsheet](https://images.datacamp.com/image/upload/v1676302629/Marketing/Blog/Seaborn_Cheat_Sheet.pdf) come in handy.


If you are satisfied with your method and you have produced a pretty visualization, please share it!

In [ ]:
corr_matrix=df.corr()
plt.figure(figsize=(10, 9))
sns.set(font_scale=1.2)
sns.heatmap(corr_matrix, cmap="BuPu")
plt.title('CORRELATION HEATMAP')

## **Pair plot**

Pair plot is a fast way to inspect relationships between features. Use seaborn's `.pairplot()` function to draw a pairplot if the first 10 columns (including price) and inspect their relationships. Set the diagonal elements to be KDE plot (setting the diagonal as "KDE" is a statistical trick useful to properly represent a probability density estimation). Be aware that the execution of this cell might take a little while.

In [ ]:
sns.pairplot(df, vars=df.columns[0:10], diag_kind='kde')

**Question**

Do you see some correlation between variables? I so, between which variables?

# **Simple Linear Regression**

## **Data preparation**

We will split the data to train and test datasets such that the test dataset is 20% of original data. Use `sklearn.model_selection.train_test_split` function to split the DataFrame to X_train and X_test.

*   **X_train** is **80%** of observation randomly chosen.
*   **X_test** is the rest **20%**.

Both X_train and X_test are `pd.DataFrame` object and include `'price'` in the table. Note that the `train_test_split` can handle DataFrames as well as arrays.

> 💡**Tip:** Since we would like X_train to be 80% of the observation and X_test to be 20% of the observations, print length of X_train and X_test.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df, test_size=0.2)
print("lenght X_train =", len(X_train))
print("lenght X_test =", len(X_test))

## **Train a simple linear regression model**

Use the best_guess_predictor as a single predictor and build a simple linear regression model using **`statsmodels.formula.api.ols`** function (https://www.statsmodels.org/dev/example_formulas.html).

Substitute the "`best_predictor`" part of the string as formula argument and run the model (trained on the X_train portion of our data). Thereafter, you can print out the result summary.

> 💡**Tip:** We had imported the library at the top of this notebook. So you can use the `smf` alias.
>
>**`import statsmodels.formula.api as smf`**

**N.B.:** It is recommended that you use the `statsmodel` library to do the regression analysis as opposed to others like `sklearn`. The `sklearn` library is great for advanced topics, but it's easier to get lost in a sea of details and it's not needed for these kind of problems.



In [ ]:
model = smf.ols(formula='price~grade', data=X_train).fit()
model.summary()

**Question**

What is the adjusted R-squared value?

## **Best predictor**

In the previous section, we picked a best guess predictor for price based on the correlation matrix. Now we will consider whether the best_guess_predictor that we used is still the best.

You now have to iteratively try the model with the other features having higher correlation value and see the summary outcome (replace string "`try_predictor`" in the cell below).

As a final task, print out a list of the top three predictors in decreasing order.

>💡**Tip:** Linear regression uses adjusted R squared as fit performance, so you can rank by this metric. Complete the `predictors_r2` dictionary taking note of adjusted R2 and then derive a ranking.



In [ ]:
# try the model with the other features with higher correlation value

model = smf.ols(formula='price~<try_predictor>', data=X_train).fit()
model.summary()

In [ ]:
# complete dictionary

'''
predictors_r2 = {
    'feature_1': 0.000,
    'feature_2': 0.000,
}
'''

# update top_three

top_three = ['feature_1','feature_2','feature_3']

**Questions**

*  What were your top three predictors?
*  How did you order your list of predictors to select those as the top ones?
*  Is your top predictor for this section the same as the best guess predictor you selected before?